## **Import Libraries**

In [35]:
import torch
import csv
import torch.nn as nn
import random
import numpy as np
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable 
from tqdm import tqdm
import random
import torch.nn.functional as F
import heapq
import wandb

## **SET DEVICE (CPU / GPU)**

In [36]:
# Set Device which to use
def set_device():
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
    return device

device = set_device()
print(device)

cuda


In [37]:
!wandb login 57566fbb0e091de2e298a4320d872f9a2b200d12

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## **LOAD DATA**

In [38]:
def load_data(lan = 'hin'):
    train_data_path = '/kaggle/input/vocabs/Dataset/hin/hin_train.csv'
    test_data_path = '/kaggle/input/vocabs/Dataset/hin/hin_test.csv'
    val_data_path = '/kaggle/input/vocabs/Dataset/hin/hin_valid.csv'
    
    train_x = []
    train_y = []
    with open(train_data_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # next(csv_reader)  # Skip the header row if it exists
        for row in csv_reader:
            train_x.append(row[0] + '$')
            train_y.append('#' + row[1] + '$')
       
    
    val_x, val_y = [], []
    with open(val_data_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # next(csv_reader)  # Skip the header row if it exists
        for row in csv_reader:
            val_x.append(row[0] + '$')
            val_y.append('#' + row[1] + '$')
    
    test_x, test_y = [], []
    with open(test_data_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # next(csv_reader)  # Skip the header row if it exists
        for row in csv_reader:
            test_x.append(row[0] + '$')
            test_y.append('#' + row[1] + '$')

    # Conversion in NP Array
    train_x, train_y = np.array(train_x), np.array(train_y)
    val_x, val_y = np.array(val_x), np.array(val_y)
    test_x, test_y = np.array(test_x), np.array(test_y)

    #get size of data
    max_decoder_length_train = np.max(np.array([len(s) for s in train_y]))
    max_decoder_length_val = np.max(np.array([len(s) for s in val_y]))
    max_decoder_length_test = np.max(np.array([len(s) for s in test_y]))
    max_decoder_length = max(max_decoder_length_train, max(max_decoder_length_val, max_decoder_length_test))

    max_encoder_length_train = np.max(np.array([len(s) for s in train_x]))
    max_encoder_length_val = np.max(np.array([len(s) for s in val_x]))
    max_encoder_length_test = np.max(np.array([len(s) for s in test_x]))
    max_encoder_length = max(max_encoder_length_train, max(max_encoder_length_val, max_encoder_length_test))


    print(max_decoder_length)
    print(max_encoder_length)

    

# Find the maximum length
    return_res = {
        "train_x" : train_x,
        "train_y" : train_y,
        "val_x" : val_x,
        "val_y" : val_y,
        "test_x" : test_x,
        "test_y" : test_y,
        "max_decoder_length" : max_decoder_length,
        "max_encoder_length" : max_encoder_length,
    }

    return return_res

In [39]:
res = load_data("hin")
tx = res["train_x"]
ty = res["train_y"]
vx = res["val_x"]
vy = res["val_y"]
tex = res["test_x"]
tey = res["test_y"]
max_decoder_length = res["max_decoder_length"]
max_encoder_length = res["max_encoder_length"]
train_len = tx.shape[0]
val_len = vx.shape[0]

22
27


In [40]:
def create_corpus(train_x, train_y, val_x, val_y, test_x, test_y):

    english_vocab = "#$abcdefghijklmnopqrstuvwxyz"
    input_corpus_dict = {}
    input_corpus_dict[''] = 0
    index = 1
    for characters in english_vocab:
        input_corpus_dict[characters] = index
        index += 1
    
    output_vocab = set()
    for word in train_y:
        for characters in word:
            output_vocab.add(characters)
    for word in val_y:
        for characters in word:
            output_vocab.add(characters)
    for word in test_y:
        for characters in word:
            output_vocab.add(characters)
    output_vocab.add('')
    sorted_output_vocab = sorted(output_vocab)

    output_corpus_dict = {}
    index = 0
    for characters in sorted_output_vocab:
        output_corpus_dict[characters] = index
        index += 1

    reversed_input_corpus = {value: key for key, value in input_corpus_dict.items()}
    reversed_output_corpus = {value: key for key, value in output_corpus_dict.items()}
    return_dict = {
        "input_corpus_length" : len(english_vocab),
        "output_corpus_length" : len(output_vocab),
        "input_corpus_dict" : input_corpus_dict,
        "output_corpus_dict" : output_corpus_dict,
        "reversed_input_corpus" : reversed_input_corpus,
        "reversed_output_corpus" : reversed_output_corpus
    }
    
    return return_dict

In [41]:
res_dict = create_corpus(tx, ty, vx, vy, tex, tey)
print(res_dict["input_corpus_length"])
print(res_dict["output_corpus_length"])
print(res_dict["input_corpus_dict"])
print(res_dict["output_corpus_dict"])
print(res_dict["reversed_input_corpus"])
print(res_dict["reversed_output_corpus"])

input_corpus_length = res_dict["input_corpus_length"]
output_corpus_length = res_dict["output_corpus_length"]
output_corpus_dict = res_dict["output_corpus_dict"]
input_corpus_dict = res_dict["input_corpus_dict"]
reversed_output_corpus = res_dict["reversed_output_corpus"]
reversed_input_corpus = res_dict["reversed_input_corpus"]
        

28
68
{'': 0, '#': 1, '$': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28}
{'': 0, '#': 1, '$': 2, 'ँ': 3, 'ं': 4, 'ः': 5, 'अ': 6, 'आ': 7, 'इ': 8, 'ई': 9, 'उ': 10, 'ऊ': 11, 'ऋ': 12, 'ए': 13, 'ऐ': 14, 'ऑ': 15, 'ओ': 16, 'औ': 17, 'क': 18, 'ख': 19, 'ग': 20, 'घ': 21, 'ङ': 22, 'च': 23, 'छ': 24, 'ज': 25, 'झ': 26, 'ञ': 27, 'ट': 28, 'ठ': 29, 'ड': 30, 'ढ': 31, 'ण': 32, 'त': 33, 'थ': 34, 'द': 35, 'ध': 36, 'न': 37, 'प': 38, 'फ': 39, 'ब': 40, 'भ': 41, 'म': 42, 'य': 43, 'र': 44, 'ल': 45, 'ळ': 46, 'व': 47, 'श': 48, 'ष': 49, 'स': 50, 'ह': 51, '़': 52, 'ऽ': 53, 'ा': 54, 'ि': 55, 'ी': 56, 'ु': 57, 'ू': 58, 'ृ': 59, 'ॅ': 60, 'े': 61, 'ै': 62, 'ॉ': 63, 'ॊ': 64, 'ो': 65, 'ौ': 66, '्': 67}
{0: '', 1: '#', 2: '$', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i', 12: 'j', 13: 'k', 14: 'l', 15: 'm', 16: 'n'

In [42]:
def create_tensor(tx, ty, vx, vy, res, dict):
    max_len = max(res["max_encoder_length"], res["max_decoder_length"])
    train_input = np.zeros((max_len, train_len), dtype = 'int64')
    train_output = np.zeros((max_len, train_len), dtype = 'int64')
    val_input = np.zeros((max_len, val_len), dtype = 'int64')
    val_output = np.zeros((max_len, val_len), dtype = 'int64')

    print(dict["input_corpus_dict"])
    print(dict["output_corpus_dict"])
    word_count = 0
    for words in tx:
        index = 0
        for chars in words:
            # print(chars)
            train_input[index, word_count] = dict["input_corpus_dict"][chars]
            index += 1
        word_count += 1

    word_count = 0
    for words in ty:
        index = 0
        for chars in words:
            # print(chars)
            train_output[index, word_count] = dict["output_corpus_dict"][chars]
            index += 1
        word_count += 1

    word_count = 0
    for words in vx:
        index = 0
        for chars in words:
            # print(chars)
            val_input[index, word_count] = dict["input_corpus_dict"][chars]
            index += 1
        word_count += 1

    word_count = 0
    for words in vy:
        index = 0
        for chars in words:
            # print(chars)
            val_output[index, word_count] = dict["output_corpus_dict"][chars]
            index += 1
        word_count += 1


    # Convert in tensor
    train_input = torch.tensor(train_input)
    train_output = torch.tensor(train_output)
    val_input = torch.tensor(val_input)
    val_output = torch.tensor(val_output)

    return_dict = {
        "train_input" : train_input,
        "train_output" : train_output,
        "val_input" : val_input,
        "val_output" : val_output,
    }
    return return_dict

In [43]:
tensors = create_tensor(tx, ty, vx, vy, res, res_dict)
train_input = tensors["train_input"]
val_input = tensors["val_input"]
train_output = tensors["train_output"]
val_output = tensors["val_output"]
print(train_input.shape)
print(train_output.shape)

{'': 0, '#': 1, '$': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28}
{'': 0, '#': 1, '$': 2, 'ँ': 3, 'ं': 4, 'ः': 5, 'अ': 6, 'आ': 7, 'इ': 8, 'ई': 9, 'उ': 10, 'ऊ': 11, 'ऋ': 12, 'ए': 13, 'ऐ': 14, 'ऑ': 15, 'ओ': 16, 'औ': 17, 'क': 18, 'ख': 19, 'ग': 20, 'घ': 21, 'ङ': 22, 'च': 23, 'छ': 24, 'ज': 25, 'झ': 26, 'ञ': 27, 'ट': 28, 'ठ': 29, 'ड': 30, 'ढ': 31, 'ण': 32, 'त': 33, 'थ': 34, 'द': 35, 'ध': 36, 'न': 37, 'प': 38, 'फ': 39, 'ब': 40, 'भ': 41, 'म': 42, 'य': 43, 'र': 44, 'ल': 45, 'ळ': 46, 'व': 47, 'श': 48, 'ष': 49, 'स': 50, 'ह': 51, '़': 52, 'ऽ': 53, 'ा': 54, 'ि': 55, 'ी': 56, 'ु': 57, 'ू': 58, 'ृ': 59, 'ॅ': 60, 'े': 61, 'ै': 62, 'ॉ': 63, 'ॊ': 64, 'ो': 65, 'ौ': 66, '्': 67}
torch.Size([27, 51200])
torch.Size([27, 51200])


## **Encoder, Decoder & Seq2Seq**

In [44]:
class Encoder(nn.Module):
    def __init__(self, PARAM):
        super(Encoder, self).__init__()
        self.input_size = PARAM["encoder_input_size"]
        self.embedding_size = PARAM["encoder_embedding_size"]
        self.hidden_size = PARAM["encoder_hidden_size"]
        self.num_layers = PARAM["encoder_num_layers"]
        self.drop_prob = PARAM["encoder_dropout_prob"]
        self.cell_name = PARAM["encoder_cell_type"]
        self.bidirectional = PARAM["bidirectional"]

        self.dropout = nn.Dropout(self.drop_prob)
        self.embedding = nn.Embedding(self.input_size, self.embedding_size)
        if self.cell_name == "LSTM":
            self.cell = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.drop_prob, bidirectional = self.bidirectional)
        if self.cell_name == "GRU":
            self.cell = nn.GRU(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.drop_prob, bidirectional = self.bidirectional)
        if self.cell_name == "RNN":
            self.cell = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.drop_prob, bidirectional = self.bidirectional)
    
    def forward(self, x):
        embedding = self.embedding(x)
        drops = self.dropout(embedding)
        if self.cell_name == "RNN" or self.cell_name == "GRU":
            cells = None
            outputs, hidden = self.cell(drops)
        elif self.cell_name == "LSTM":
            outputs, (hidden, cells)= self.cell(drops)
        return hidden, cells
    
class Decoder(nn.Module):
    def __init__(self, PARAM):
        super(Decoder, self).__init__()
        self.input_size = PARAM["decoder_input_size"]
        self.embedding_size = PARAM["decoder_embedding_size"]
        self.hidden_size = PARAM["decoder_hidden_size"]
        self.output_size = PARAM["decoder_output_size"]
        self.num_layers = PARAM["decoder_num_layers"]
        self.drop_prob = PARAM["decoder_dropout_prob"]
        self.cell_name = PARAM["decoder_cell_type"]
        self.bidirectional = PARAM["bidirectional"]
        self.dropout = nn.Dropout(self.drop_prob)
        self.embedding = nn.Embedding(self.input_size, self.embedding_size)
        
        if self.cell_name == "LSTM":
            self.cell = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.drop_prob, bidirectional = self.bidirectional)
        if self.cell_name == "GRU":
            self.cell = nn.GRU(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.drop_prob, bidirectional = self.bidirectional)
        if self.cell_name == "RNN":
            self.cell = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.drop_prob, bidirectional = self.bidirectional)
            
        if self.bidirectional:
            self.fc = nn.Linear(self.hidden_size*2, self.output_size)
        else:
            self.fc = nn.Linear(self.hidden_size, self.output_size)
    
    def forward(self, x, hidden, cells):
        x = x.unsqueeze(0)
        embedding = self.embedding(x)
        drops = self.dropout(embedding)
        if self.cell_name == "RNN" or self.cell_name == "GRU":
            outputs, hidden = self.cell(drops, hidden)
        elif self.cell_name == "LSTM":
            outputs, (hidden, cells) = self.cell(drops, (hidden, cells))
        predictions = self.fc(outputs).squeeze(0)
        predictions = F.log_softmax(predictions, dim = 1)
        return predictions, hidden, cells
    
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, param):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.teacher_forcing_ratio = param["tfr"]
    
    def forward(self, src, target):
        batch_size = src.shape[1]
        target_len = target.shape[0]
        
        target_vocab_size = len(output_corpus_dict)
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        encoder_hidden, cell = self.encoder(src)

        x = target[0]
        
        for tar in range(1, target_len):
            output, encoder_hidden, cell = self.decoder(x, encoder_hidden, cell)
            outputs[tar] = output

            if random.random() < self.teacher_forcing_ratio:
                x = target[tar]
            else:
                x = output.argmax(dim=1)
        
        return outputs

In [45]:
# def predict(model, PARAM):
#     """
#     Predict the output sequence given an input word using the trained model.

#     Args:
#         model (nn.Module): The trained model.
#         PARAM (dict): Dictionary containing necessary parameters including:
#             - "word" (str): Input word to be translated.
#             - "input_corpus_dict" (dict): Dictionary mapping input characters to indices.
#             - "output_corpus_dict" (dict): Dictionary mapping output indices to characters.
#             - "reversed_output_corpus" (dict): Dictionary mapping output characters to indices.

#     Returns:
#         str: Predicted output sequence.
#     """
#     word = PARAM["word"]
#     input_char_index = PARAM["input_corpus_dict"]
#     output_char_index = PARAM["output_corpus_dict"]
#     reverse_target_char_index = PARAM["reversed_output_corpus"]
    
#     data = np.zeros((len(input_char_index), 1), dtype=int)
#     word_t = ''
#     t_z = 0
    
#     for t, char in enumerate(word):
#         data[t, 0] = input_char_index.get(char, input_char_index['$'])
#     t_z = t + 1   
#     data[t_z:, 0] = input_char_index['$']
    
#     data = torch.tensor(data, dtype=torch.int64).to(device)

#     with torch.no_grad():
#         hidden, cell = model.encoder(data)
    
#     output_char = output_char_index['#']    
#     out_chr_reshape = np.array(output_char).reshape(1,)    
#     x = torch.tensor(out_chr_reshape).to(device)

#     for _ in range(1, len(output_char_index)):
#         output, hidden, cell = model.decoder(x, hidden, cell)
#         predicted_index = output.argmax(1).item()
#         predicted_char = reverse_target_char_index.get(predicted_index, '$')
        
#         if predicted_char != '$':
#             word_t += predicted_char
#         else:
#             break

#     return word_t

# # Define the input sequence
# # PARAM_PRED = {
# #     "word" : "aditya",
# #     "input_corpus_dict" : input_corpus_dict,
# #     "output_corpus_dict" : output_corpus_dict,
# #     "reversed_output_corpus" : reversed_output_corpus,
# # }

# # lst = ["harsh", "ayush", "aditya", "raghav", "antim", "tanish", "priyakant"]
# # for word in lst:
# #     PARAM_PRED['word'] = word
# #     output_sequence = predict(model, PARAM_PRED)
# #     print(PARAM_PRED["word"], end = " ==> ")
# #     print(output_sequence)

## **Setting Optimizer**

In [46]:
def set_optimizer(name, model, learning_rate):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    if name == "adam":
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    if name == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    if name == "rmsprop":
        optimizer = optim.RMSprop(model.parameters(), lr = learning_rate)
    if name == "adagrad":
        optimizer = optim.Adagrad(model.parameters(), lr = learning_rate)
    return optimizer

## **BEAM SEARCH**

In [47]:
def beam_search(PARAM, model, word, device, max_encoder_length, input_corpus_dict, output_corpus_dict, reverse_output_corpus):
    dtype = torch.int32
    data, word_t = np.zeros((max_encoder_length, 1), dtype=np.int32), ""
    index = 0
    for character in word:
        data[index, 0] = input_corpus_dict[character]
        index += 1
    data[index, 0] = input_corpus_dict['$']
    
    data = torch.tensor(data, dtype=torch.int32).to(device)

    with torch.no_grad():
        hidden, cell = model.encoder(data)
    
    output_start = output_corpus_dict['#']
    out_reshape = np.array(output_start).reshape(1,)
    hidden_par = hidden.unsqueeze(0)
    initial_sequence = torch.tensor(out_reshape).to(device)
    beam = [(0.0, initial_sequence, hidden_par)]

    for i in range(len(output_corpus_dict)):
        candidates = []
        for score, seq, hidden in beam:
            if seq[-1].item() == output_corpus_dict['$']:
                candidates.append((score, seq, hidden))
                continue
            
            reshape_last = np.array(seq[-1].item()).reshape(1, )
            hdn = hidden.squeeze(0) 
            x = torch.tensor(reshape_last).to(device)
            output, hidden, cell = model.decoder(x, hdn, cell)
            probabilities = F.softmax(output, dim=1)
            topk_probs, topk_tokens = torch.topk(probabilities, k = PARAM["beam_width"])               
        
            for prob, token in zip(topk_probs[0], topk_tokens[0]):
                new_seq = torch.cat((seq, token.unsqueeze(0)), dim=0)
                ln_ns = len(new_seq)
                ln_pf = ((ln_ns - 1) / 5)
                candidate_score = score + torch.log(prob).item() / (ln_pf ** PARAM["length_panelty"])
                candidates.append((candidate_score, new_seq, hidden.unsqueeze(0)))
        
        beam = heapq.nlargest(PARAM["beam_width"], candidates, key=lambda x: x[0])
    
    best_score, best_sequence, _ = max(beam, key=lambda x: x[0]) 
    
    word_t = ''.join([reverse_output_corpus[token.item()] for token in best_sequence[1:]])
    
    return word_t[:-1]
            
  

In [48]:
def run_epoch(model, train_batches_x, train_batches_y, optimizer, loss_function):
    total_words = 0
    correct_pred = 0
    total_loss = 0
    model.train()

    with tqdm(total=len(train_batches_x), desc='Training') as pbar:
        for i, ( x_batch, y_batch) in enumerate(zip(train_batches_x, train_batches_y)):
            target_batch, input_batch = y_batch.to(device), x_batch.to(device)
            optimizer.zero_grad()
            output_batch = model(input_batch, target_batch)
            mask = (target_batch != output_corpus_dict[''])  # Padding mask
            non_pad_targets = target_batch[mask]
            non_pad_outputs = output_batch[mask].reshape(-1, output_batch.shape[2])
            loss = loss_function(non_pad_outputs, non_pad_targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
            optimizer.step()
            total_words += non_pad_targets.size(0)
            correct_pred += torch.sum(torch.argmax(non_pad_outputs, dim=1) == non_pad_targets).item()
            total_loss += loss.item()
            pbar.update(1)
    accuracy = correct_pred / total_words
    avg_loss = total_loss / len(train_batches_x)  # Average loss per batch
    
    return accuracy, avg_loss

def evaluate_using_char(model, val_batches_x, val_batches_y, loss_function):
    model.eval()
    with torch.no_grad():
        val_total_loss = 0
        val_total_words = 0
        val_correct_pred = 0

        with tqdm(total=len(val_batches_x), desc='Validation') as pbar:
            for i, (x_val_batch, y_val_batch) in enumerate(zip(val_batches_x, val_batches_y)):
                target_val_batch, input_val_batch = y_val_batch.to(device), x_val_batch.to(device)
                output_val_batch = model(input_val_batch, target_val_batch)
                mask = (target_val_batch != output_corpus_dict[''])  # Padding mask
                non_pad_targets_val = target_val_batch[mask]
                non_pad_outputs_val = output_val_batch[mask].reshape(-1, output_val_batch.shape[2])
                val_loss = loss_function(non_pad_outputs_val, non_pad_targets_val)
                val_total_loss += val_loss.item()
                val_total_words += non_pad_targets_val.size(0)
                val_correct_pred += torch.sum(torch.argmax(non_pad_outputs_val, dim=1) == non_pad_targets_val).item()
                pbar.update(1)

    val_accuracy = val_correct_pred / val_total_words
    val_avg_loss = val_total_loss / len(val_batches_x)
        
    return val_accuracy, val_avg_loss

def evaluate_using_words(PARAM, model, val_input, vx, vy):
    model.eval()
    with torch.no_grad():
        val_total_words_beam = 0
        val_correct_pred_beam = 0
        with tqdm(total=val_input.shape[1], desc='Validation_With_Beam_Search') as pbar:
            for word, target_word in zip(vx, vy):
                val_total_words_beam += 1
                predicted_word = beam_search(PARAM, model, word, device, max_encoder_length, input_corpus_dict, output_corpus_dict, reversed_output_corpus)
                if predicted_word == target_word[1:-1]:
                    val_correct_pred_beam += 1
                pbar.update(1)

    val_accuracy_beam = val_correct_pred_beam / val_total_words_beam
        
    return val_accuracy_beam, val_correct_pred_beam
    
    



## **Train Using Beam Search**

In [49]:
HYPER_PARAM = {
    "encoder_input_size": len(input_corpus_dict),
    "encoder_embedding_size": 256,
    "encoder_hidden_size": 512,
    "encoder_num_layers": 3,
    "encoder_dropout_prob": 0.1,
    "encoder_cell_type": "LSTM",
    "decoder_input_size": len(output_corpus_dict),
    "decoder_embedding_size": 256,
    "decoder_hidden_size": 512,
    "decoder_output_size": len(output_corpus_dict),
    "decoder_num_layers": 3,
    "decoder_dropout_prob": 0.1,
    "decoder_cell_type": "LSTM",
    "beam_width" : 1,
    "length_panelty" : 0.6,
    "bidirectional" : True,
    "learning_rate" : 0.001,
    "batch_size" : 64,
    "epochs" : 1,
    "optimizer" : "adam",
    "tfr" : 0.5,
}

def training(PARAM, wandb_log = 0):
    if wandb_log == 1:
        wandb.init(project='DL-Assignment3')
        wandb.run.name = 'Testing'
    learning_rate = PARAM["learning_rate"]
    epochs = PARAM["epochs"]
    batch_size = PARAM["batch_size"]

    encoder = Encoder(PARAM).to(device)
    decoder = Decoder(PARAM).to(device)

#     # Initialize model
    model = Seq2Seq(encoder, decoder, PARAM).to(device)
    print(model)

    # Define loss function and optimizer
    loss_function = nn.CrossEntropyLoss()
    optimizer = set_optimizer(PARAM["optimizer"], model, learning_rate)

    # Split dataset into batches
    train_batches_x = torch.split(train_input, batch_size, dim=1)
    train_batches_y = torch.split(train_output, batch_size, dim=1)
    val_batches_x = torch.split(val_input, batch_size, dim=1)
    val_batches_y = torch.split(val_output, batch_size, dim=1)

    # Training loop
    for epoch in range(epochs):
        print(f"Epoch :: {epoch+1}/{epochs}")
        
        accuracy, avg_loss = run_epoch(model, train_batches_x, train_batches_y, optimizer, loss_function)  # Average loss per batch

        val_accuracy, val_avg_loss = evaluate_using_char(model, val_batches_x, val_batches_y, loss_function)

        val_accuracy_beam, val_correct_pred_beam = evaluate_using_words(PARAM, model, val_input, vx, vy)
        print(f"Epoch : {epoch+1} Train Accuracy: {accuracy*100:.4f}, Train Loss: {avg_loss:.4f}\nValidation Accuracy: {val_accuracy*100:.4f}, Validation Loss: {val_avg_loss:.4f}, \nValidation Acc. With BeamSearch: {val_accuracy_beam*100:.4f}, Correctly Predicted : {val_correct_pred_beam}/{val_input.shape[1]}")
        if wandb_log:
            wandb.log(
                    {
                        'epoch': epoch+1,
                        'training_loss' : avg_loss,
                        'training_accuracy' : accuracy,
                        'validation_loss' : val_avg_loss,
                        'validation_accuracy_using_char' : val_accuracy,
                        'validation_accuracy_using_word' : val_accuracy_beam,
                        'correctly_predicted' : val_correct_pred_beam
                    }
                )
    if wandb_log == 1:
        wandb.finish()
    return model, val_accuracy_beam

In [51]:
model, acc = training(HYPER_PARAM, wandb_log = 0)

sentance = "Jai shree krishna"
sentance = sentance.lower()
lst = sentance.split(" ")
print(type(lst))
for word in lst:
    output_sequence = beam_search(HYPER_PARAM, model, word ,device, max_encoder_length, input_corpus_dict, output_corpus_dict, reversed_output_corpus)
    print(output_sequence, end = " ")

Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.1, inplace=False)
    (embedding): Embedding(29, 256)
    (cell): LSTM(256, 512, num_layers=3, dropout=0.1, bidirectional=True)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.1, inplace=False)
    (embedding): Embedding(68, 256)
    (cell): LSTM(256, 512, num_layers=3, dropout=0.1, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=68, bias=True)
  )
)
Epoch :: 0/1


Validation_With_Beam_Search: 100%|██████████| 4096/4096 [00:47<00:00, 86.76it/s]


Epoch : 1 Train Accuracy: 55.7133, Train Loss: 1.5694
Validation Accuracy: 67.4065, Validation Loss: 1.1684, 
Validation Acc. With BeamSearch: 27.4414, Correctly Predicted : 1124/4096
<class 'list'>
ज़ई श्रे कृष्णा 

In [52]:
sentance = "paritraanaaya sadhuunaam vinaashaaya chadushkritaam dharma samsthaapanaarthaaya sambhavaami yuge yuge"
sentance = sentance.lower()
lst = sentance.split(" ")
print(type(lst))
for word in lst:
    output_sequence = beam_search(HYPER_PARAM, model, word ,device, max_encoder_length, input_corpus_dict, output_corpus_dict, reversed_output_corpus)
    print(output_sequence, end = " ")

<class 'list'>
परित्रणायय सधुऊनम विनाशायय चादुष्कृततम धर्मा समस्थापाराथ्या संभावामी युगे युगे एएए 

In [ ]:
sweep_config = {
            'name': 'sweep random 1',
            'method': 'random',
            'metric': { 'goal': 'maximize','name': 'Accuracy'},
            'parameters': 
                {
                    'epochs': {'values': [10]},
                    'cell_type': {'values': ['RNN', 'LSTM', 'GRU']},
                    'embedding_size': {'values': [128, 256, 512]},
                    'hidden_size': {'values': [128, 256, 512, 1024]},
                    'num_layers': {'values': [1, 2, 3]},
                    'dropout': {'values': [0.3, 0.5, 0.7]},
                    'optimizer' : {'values' : ['adam', 'sgd', 'rmsprop', 'adagrad']},
                    'learning_rate': {'values': [0.001, 0.005, 0.01, 0.1]},
                    'batch_size': {'values': [32, 64]},
                    'teacher_fr' : {'values': [0.3, 0.5, 0.7]},
                    'length_penalty' : {'values': [0.4, 0.5, 0.6]},
                    'bi_dir' : {'values': [True, False]},
                    'beam_width': {'values': [1, 2, 3]}
                }
            }

In [ ]:
def train():
    var1 = wandb.init(project="DL-Assignment3")
    var2 = var1.config
   
    wandb.run.name = (f"cell_type:{var2.cell_type}_epochs:{var2.epochs}_lr:{var2.learning_rate}_batch_size:{var2.batch_size}_beam_width:{var2.beam_width}_opt:{var2.optimizer}_dropout:{var2.dropout}_teacher_fr:{var2.teacher_fr}_embadding_size:{var2.embedding_size}")

    HYPER_PARAM = {
    "encoder_input_size": len(input_corpus_dict),
    "encoder_embedding_size": var2.embedding_size,
    "encoder_hidden_size": var2.hidden_size,
    "encoder_num_layers": var2.num_layers,
    "encoder_dropout_prob": var2.dropout,
    "encoder_cell_type": var2.cell_type,
    "decoder_input_size": len(output_corpus_dict),
    "decoder_embedding_size": var2.embedding_size,
    "decoder_hidden_size": var2.hidden_size,
    "decoder_output_size": len(output_corpus_dict),
    "decoder_num_layers": var2.num_layers,
    "decoder_dropout_prob": var2.dropout,
    "decoder_cell_type": var2.cell_type,
    "beam_width" : var2.beam_width,
    "length_panelty" : var2.length_penalty,
    "bidirectional" : var2.bi_dir,
    "learning_rate" : var2.learning_rate,
    "batch_size" : var2.batch_size,
    "epochs" : var2.epochs,
    "optimizer" : var2.optimizer,
    "tfr" : var2.teacher_fr,
}

    model, accuracy = training(HYPER_PARAM, wandb_log = 1)
    wandb.log({
                "Accuracy" : accuracy
            })

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="DL-Assignment3")
wandb.agent(sweep_id, train)
wandb.finish()